---

<center><b> 개요 </b> :  본 대회는 다양한 장비/서비스에서 일어나는 시스템 데이터를 통해 사용자의 <u><b>불편을 예지</b></u>하기 위해<br>‘시스템 데이터’와 ‘사용자 불편 발생 데이터’를 분석하여 <u><b>불편을 느낀 사용자와 불편 요인들</b></u>을 찾는 대회입니다.<br><br></center>

---

# <center>EDA 목차</center>

* `Index`
    1. 패키지 불러오기
    2. 데이터 불러오기
    3. 데이터 정보  
        - 3-1. 데이터 소개
        - 3-2. 결측치 확인
        - 3-3. 데이터 기본 전처리
        - 3-3. 이상치 확인
    4. 불만 제기 시간 확인  
        - 4-1. 고객들의 시간별 불만 제기
    5. Err Data 관계 해석
        - 5-1. Errtype
        - 5-2. Errcode
        - 5-3. Fwver
        - 5-4. Model
    6. Quality Data 수치 해석
        - 6-1. Quality 값
        - 6-2. Fwver
    7. Err Data와 Quality Data 관계 해석
    8. 결과 정리

---

#  <center>1. 패키지 불러오기</center>

|라이브러리|버전|
|-------|---|
|OS|macOS Catalina|
|python|3.8.3|
|numpy|1.20.0|
|pandas|1.2.1|
|tqdm|4.56.0|

---

In [130]:
import os
import random # Analysis
import datetime as dt # Analysis
import numpy as np # Analysis
import pandas as pd # Analysis

from tqdm import tqdm

In [131]:
import warnings
warnings.filterwarnings(action='ignore') # ignore warnings

In [132]:
# datafarme 보여주는 범위 설정

pd.options.display.max_columns=1000
pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format

---

#  <center>2. 데이터 불러오기</center>

---

In [4]:
load_path = './data/' # 데이터 불러오는 경로
save_path = './preprocessing/' # 전처리 이후 데이터 저장하는 경로

In [5]:
train_err = pd.read_csv(load_path + 'train_err_data.csv')
train_qual = pd.read_csv(load_path + 'train_quality_data.csv')
train_prob = pd.read_csv(load_path + 'train_problem_data.csv')

In [6]:
test_err = pd.read_csv(load_path + 'test_err_data.csv')
test_qual = pd.read_csv(load_path + 'test_quality_data.csv')

---

#  <center>3. 데이터 정보</center>

---

## 3-1. 데이터 소개

* `Err Data`는 사람들이 에러를 접한 시간을 기준으로  
  <u><b>어떤 Model</b> & <b>Fwver을 사용</b>하고, <b>접한 Errtype</b> & <b>Errcode는 무엇인지</b></u>에 대하여 알려주는 데이터이다.

* 변수는 총 6개이고, 관측치는 약 1600만개로 User ID별로 시계열 데이터가 나와있는 자료이다.  
    * User ID : 사용자 고유 ID
    * Time : 에러 발생 시간
    * Model : 에러가 발생한 모델명
    * Fwver : 에러가 발생한 펌웨어 버전
    * Errtype : 에러 분류 (에러 타입)
    * Errcode : 어떤 에러가 발생하였는지 (에러 코드)

In [7]:
train_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [8]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 756.8+ MB


* `Quality Data`는 사용자의 시스템 작동 중 문제가 발생하면  
  측정 가능한 지표들로 <b><u>해당 시점으로부터 2시간 단위로 수집한 정보</u></b>를 알려주는 데이터이다.
    * User ID : 사용자 고유 ID
    * Time : 퀄리티가 수집되기 시작한 시간
    * Fwver : 퀄리티 시작 시점 기준의 펌웨어 버전
    * Quality : 에러 퀄리티 수치 (0~12, 총 13개 컬럼)

In [9]:
train_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [10]:
test_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


* `Problem Data`는 전체 사용자 중에서, <b>불만을 제기한 사람들의 시간</b>을 알려주는 데이터이다.
    * User ID : 불만을 제기한 유저
    * Time : 불만을 제기한 시간
    * <u>한 사용자가 여러번 불만을 제기할 수 있음</u>

In [11]:
train_prob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


-----

## 3-2. 결측치 확인

1. `Train Err Data` Missing Value  
    + Errcode 결측치 존재

In [12]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [13]:
train_err[train_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN


* 동일한 유저, 동일한 시간, 동일한 펌웨어버전, 동일한 에러타입의 경우 → <u><b>같은 에러 코드를 가질 확률이 높음</b></u>을 이용 → `결측치 대체`

In [14]:
tre = train_err.iloc[:4000000,:].copy()
tre_missing = tre.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame()
tre_missing['errcode'] = tre_missing['errcode'].apply(lambda x : len(x))

ltwo = len(tre_missing[tre_missing['errcode'] != 1])
lone = len(tre_missing[tre_missing['errcode'] == 1])
print("동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : 0.039%


In [15]:
train_err[(train_err['user_id'] == 13639) & (train_err['fwver'] == '04.33.1261') &
          (train_err['errtype'] == 5) & (train_err['time'] == 20201121191718)]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN
3825745,13639,20201121191718,model_2,04.33.1261,5,40013


* 따라서 동일한 에러 코드로 결측치 교체

In [16]:
train_err['errcode'] = train_err['errcode'].fillna('40013')

---

2. `Train Quality Data` Missing Value  
    + Fwver 결측치 존재
    + quality_0, quality_2, quality_5 결측치 존재

In [17]:
train_qual.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

* NaN값을 갖는 경우에는 Quality가 지속적으로 `오랜` 기간동안 측정됨을 알 수 있다
* 반대로, NaN값을 갖지 않는 경우에는 상대적으로 적은 기간동안 측정됨을 알 수 있다
* 따라서, NaN값을 갖는 경우는 '이상치'에 해당함으로, 해당 결측치는 제거한다.

In [18]:
# quality가 측정된 기간의 길이 (NaN값의 경우)
train_qual[train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

55.45762711864407

In [19]:
train_qual[~train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

7.9863813229571985

In [20]:
fwms_idx = train_qual[train_qual['fwver'].isnull()].index
train_qual = train_qual.drop(fwms_idx)

* Quality_0, Quality_2, Quality_5 결측치

In [21]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].fillna(train_qual[i].mode(0)[0])

---

3. `Test Err Data` Missing Value

In [22]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [23]:
test_err[test_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,20201115044317,model_2,04.33.1261,5,NaN
4038892,33681,20201103110259,model_2,04.33.1185,5,NaN
9486881,38991,20201127213838,model_2,04.33.1261,5,NaN
10425473,39894,20201128144712,model_1,04.16.3553,5,NaN


* 동일한 유저, 동일한 시간, 동일한 펌웨어버전, 동일한 에러 타입의 경우 → <u><b>같은 에러 코드를 가질 확률이 높음</b></u>을 이용

In [24]:
te = test_err.iloc[:4000000,:].copy()
te_missing = te.groupby(['user_id','time','fwver','errtype'])['errcode'].unique().to_frame()
te_missing['errcode'] = te_missing['errcode'].apply(lambda x : len(x))

ltwo = len(te_missing[te_missing['errcode'] != 1])
lone = len(te_missing[te_missing['errcode'] == 1])
print("동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : %.3f%%"%(ltwo/lone * 100))

동일한 특성을 갖는 Row안에 다른 Err Code가 발생할 확률 : 0.054%


* 따라서 동일한 에러 코드로 결측치 교체

In [25]:
test_err.iloc[937967,5] = '40053'

In [26]:
test_err.iloc[4038892,5] = '40053'

In [27]:
test_err.iloc[9486881,5] = '40053'

In [28]:
test_err.iloc[10425473,5] = '-1010'

* Train Err User는 15000명, Test Err User는 14999명  
    + 임의의 유저 1명 추가 (최빈값 대체)

In [29]:
missing_value = pd.Series([43262, train_err.time.mode()[0], train_err.model_nm.mode()[0],
                           train_err.fwver.mode()[0], train_err.errtype.mode()[0],
                           train_err.errcode.mode()[0]], index = test_err.columns)

In [30]:
test_err = test_err.append(missing_value, ignore_index = True)

---

4. `Test Quality Data` Missing Value  
    + Fwver 결측치 존재
    + quality_0, quality_1, quality_2, quality_5 결측치 존재

In [31]:
test_qual.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

* NaN값을 갖는 경우에는 Quality가 지속적으로 `오랜` 기간동안 측정됨을 알 수 있다
* 반대로, NaN값을 갖지 않는 경우에는 상대적으로 적은 기간동안 측정됨을 알 수 있다
* 따라서, NaN값을 갖는 경우는 '이상치'에 해당함으로, 해당 결측치는 제거한다.

In [32]:
# quality가 측정된 기간의 길이 (NaN값의 경우)
test_qual[test_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

36.627450980392155

In [33]:
test_qual[~test_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

7.348095875410634

In [34]:
fwms_idx = test_qual[test_qual['fwver'].isnull()].index
test_qual = test_qual.drop(fwms_idx)

* Qaulity_0, Quality_1, Quality_2, Quality_5 결측치

In [35]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].fillna(test_qual[i].mode(0)[0])

* 결측치 제대로 제거(또는 대체)되었는지 확인

In [36]:
print(train_err.isnull().sum().sum(), end=", ")
print(test_err.isnull().sum().sum(), end=", ")
print(train_qual.isnull().sum().sum(), end=", ")
print(test_qual.isnull().sum().sum())

0, 0, 0, 0


---

## 3-3. 데이터 기본 전처리

1. <b>`변수 타입`</b> 통일

* `time` 변수의 타입을 <b>`int`</b>에서 <b>`datetime`</b>으로 변경

In [37]:
# time 변수 타입 변경 이전 데이터 저장
train_err.to_csv(save_path + "train_err_data.csv", index=False)
train_qual.to_csv(save_path + "train_quality_data.csv", index=False)
train_prob.to_csv(save_path + "train_problem_data.csv", index=False)

test_err.to_csv(save_path + "test_err_data.csv", index=False)
test_qual.to_csv(save_path + "test_quality_data.csv", index=False)

In [38]:
def make_datetime(x): # datetime 데이터로 변환
        x = str(x)
        year  = int(x[:4])
        month = int(x[4:6])
        day   = int(x[6:8])
        hour  = int(x[8:10])
        minute  = int(x[10:12])
        sec  = int(x[12:])
        return dt.datetime(year, month, day, hour, minute, sec)

In [39]:
train_err.time = train_err.time.apply(lambda x : make_datetime(x))

In [40]:
train_qual = train_qual.sort_values(['user_id','time']).reset_index(drop=True)
train_qual.time = train_qual.time.apply(lambda x : make_datetime(x))

In [41]:
train_prob = train_prob.sort_values(['user_id','time']).reset_index(drop=True)
train_prob.time = train_prob.time.apply(lambda x : make_datetime(x))

In [42]:
test_err.time = test_err.time.apply(lambda x : make_datetime(x))

In [43]:
test_qual.time = test_qual.time.apply(lambda x : make_datetime(x))

In [44]:
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
1,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
2,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
3,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
4,2020-11-29 09:00:00,10000,05.15.2138,0.00000,0,0.00000,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788539,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,1,0,0,0,0,17,0,0
788540,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,0,0,0,0,0,17,0,0
788541,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,3,0,0,0,0,17,0,0
788542,2020-11-24 03:30:00,24997,04.22.1778,0.00000,0,0.00000,0,0,0,0,0,0,0,17,0,0


* quality 변수의 타입을 <b>`object(또는 float)`</b>에서 <b>`int`</b>로 변경

In [45]:
def str2int(x):
    if type(x) == str:
        x = x.replace(",","")
        x = int(x)
        return x
    else:
        x = int(x)
        return x

In [46]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].apply(lambda x : str2int(x))

In [47]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].apply(lambda x : str2int(x))

---

## 3-4. 이상치 확인

###### 1. Fwver의 이상치 확인

* 펌웨어 버전의 운용 기간이 <u><b>비정상적으로 짧은 것</b></u>들이 존재하였음  
&nbsp;→&nbsp; 해당 펌웨어 버전을 사용한 유저는 펌웨어 버전당 `한 명`뿐이었음  
&nbsp;→&nbsp; 따라서, 이러한 버전을 사용한 유저는 `이상치`에 속한다고 보고 데이터를 확인하였음

In [48]:
temp = train_err[['time','fwver']].groupby("fwver").time.unique()
df = pd.DataFrame(columns = ['fwver','start','end'])
for i in range(len(temp)):
    temp.values[i].sort()
    df=df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
                 ignore_index = True)

In [49]:
def find_abnormal(x): # x는 살펴볼 기간 차이
    for fwver, st, ed in tqdm(df.values):
        if((ed - st).days < x):
            user = train_err[train_err['fwver'] == fwver].user_id.unique()
            print("fwver : {}, per_num : {}, period : {}, id : {}".format(fwver,len(user),(ed-st).days,user))

In [50]:
find_abnormal(3) # fwver : 펌웨어 버전 # per_num : 사용한 유저의 수 # id : 사용한 유저의 id

 30%|██▉       | 11/37 [00:00<00:01, 14.16it/s]

fwver : 04.22.1656, per_num : 1, period : 0, id : [19831]


 73%|███████▎  | 27/37 [00:01<00:00, 18.15it/s]

fwver : 05.15.2090, per_num : 1, period : 0, id : [18142]


100%|██████████| 37/37 [00:02<00:00, 16.10it/s]

fwver : 05.15.2092, per_num : 1, period : 0, id : [24279]


* 18142번 유저는 약 3분동안 혼자 05.15.2090 버전을 사용하였음 → 퀄리티값에 이상이 없음, 에러 로그 존재함 → 불만 제기 안함
* 19831번 유저는 하루~이틀동안 혼자 04.22.1656 버전을 사용하였음 → 퀄리티값에 이상이 있음, 에러 로그 존재함 → 불만 제기함
* 24279번 유저는 약 18분동안 혼자 05.15.2092 버전을 사용하였음 → 퀄리티값에 이상이 없음, 에러 로그 존재함 → 불만 제기함

In [51]:
abnormal_lst = [18142, 19831, 24279]
display(train_prob[train_prob['user_id'].isin(abnormal_lst)]) # 해당 유저들이 불만을 제기했는지 확인
# display(train_qual[train_qual['user_id'].isin(abnormal_lst)]) # 해당 유저들의 quality값에 이상이 있는지 확인
# display(train_err[train_err['user_id'].isin(abnormal_lst)][:50]) # 해당 유저의 Err Data 확인

,user_id,time
3499,19831,2020-11-23 11:00:00
5180,24279,2020-11-13 11:00:00


그러므로 <b>24279번 유저는</b> 이상치라고 생각할 수 있다.<br>
<span style="color:gray">(이는 추후 분석을 통해 더욱 이상치로 볼 수 있음을 확인할 수 있다, 따라서 추후 분석 이전 단계에서는 제거하지 않는다.)</span>

---

###### 2. Err Log 이상치 확인

* 에러 로그양이 적음에도 불구하고, 불만을 제기한 사람들이 존재함

In [52]:
log_lst = list(train_err.groupby('user_id')['errtype'].count().to_frame().\
               sort_values(by='errtype').reset_index().user_id.unique())[:100] # 에러 로그양이 가장 적은 유저 100명

print(len(train_prob[train_prob['user_id'].isin(log_lst)])) # 에러 로그양이 적은 유저 100명중, 불만을 제기한 유저의 수
print(list(train_prob[train_prob['user_id'].isin(log_lst)].user_id.unique())) # 에러 로그양이 적은 유저 100명 리스트

5
[12623, 16980, 20271, 20300, 21040]


* 해당 사람들의 Err Data & Quality Data를 살펴봄

In [53]:
display(train_err[train_err['user_id'] == 16980])
display(train_qual[train_qual['user_id'] == 16980])
print("\n")
display(train_err[train_err['user_id'] == 20271])
display(train_qual[train_qual['user_id'] == 20271])
print("\n")
display(train_err[train_err['user_id'] == 12623])
display(train_qual[train_qual['user_id'] == 12623])
print("\n")
display(train_err[train_err['user_id'] == 21040])
display(train_qual[train_qual['user_id'] == 21040])
print("\n")
display(train_err[train_err['user_id'] == 20300])
display(train_qual[train_qual['user_id'] == 20300])

,user_id,time,model_nm,fwver,errtype,errcode
7787926,16980,2020-11-17 22:44:56,model_6,10,5,S-61001
7787927,16980,2020-11-17 22:44:59,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
11112255,20271,2020-11-04 22:11:48,model_6,10,5,S-61001
11112256,20271,2020-11-05 21:53:00,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
2750266,12623,2020-11-04 21:12:52,model_6,10,5,S-61001
2750267,12623,2020-11-04 21:13:52,model_6,10,5,S-61001


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
12210417,21040,2020-11-04 22:43:14,model_6,10,5,B-A8002
12210418,21040,2020-11-04 22:43:47,model_6,10,5,B-A8002


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


,user_id,time,model_nm,fwver,errtype,errcode
11144486,20300,2020-11-07 20:04:39,model_3,05.15.2138,26,1
11144487,20300,2020-11-07 20:04:46,model_3,05.15.2138,12,1
11144488,20300,2020-11-07 20:04:46,model_3,05.15.2138,11,1
11144489,20300,2020-11-07 21:38:48,model_3,05.15.2138,4,0


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


에러 로그가 적은 유저들은 <b>퀄리티가 측정되지 않았다는 공통점이</b> 발견되었다.<br>
<span style="color:gray">(이는 추후 분석을 통해 더욱 이상치로 볼 수 있음을 확인할 수 있다, 따라서 추후 분석 이전 단계에서는 제거하지 않는다.) <br>(또한 이후 Err Data와 Quality Data 간의 관계분석에 사용 가능하다.)</span>

---

* 전처리 데이터 저장

In [54]:
# time 변수 타입 변경 이전 + 그외에는 전처리된(결측치, 이상치 처리) 데이터 저장
train_err.to_csv(save_path + "train_err_data_time.csv", index=False)
train_qual.to_csv(save_path + "train_quality_data_time.csv", index=False)
train_prob.to_csv(save_path + "train_problem_data_time.csv", index=False)

test_err.to_csv(save_path + "test_err_data_time.csv", index=False)
test_qual.to_csv(save_path + "test_quality_data_time.csv", index=False)

---

# 4. 불만 제기 시간 확인

---

## 4-1. 고객들이 `언제`, `무슨 요일`, `어느 시간`에 불만을 제기하였는지 확인

In [133]:
# Datetime으로 Type변경이 되기전 데이터 로드
train_err = pd.read_csv(save_path+"train_err_data.csv")
train_qual = pd.read_csv(save_path+"train_quality_data.csv")
train_prob = pd.read_csv(save_path+"train_problem_data.csv")

test_err = pd.read_csv(save_path+"test_err_data.csv")
test_qual = pd.read_csv(save_path+"test_quality_data.csv")

In [134]:
# 불만 제기 고객 확인
problem_user = train_prob.user_id.unique()
no_problem_user = list(set(train_err.user_id.unique()) - set(problem_user))

In [135]:
def make_weekday(col): # 0 : 월요일, 1: 화요일, 2: 수요일, 3: 목요일, 4: 금요일, 5: 토요일, 6: 일요일
    col = str(col)
    year  = int(col[:4])
    month = int(col[4:6])
    day   = int(col[6:8])
    weekday = dt.datetime(year,month,day).weekday()
    
#     if weekday == 0:
#         return "월요일"
#     elif weekday == 1:
#         return "화요일"
#     elif weekday == 2:
#         return "수요일"
#     elif weekday == 3:
#         return "목요일"
#     elif weekday == 4:
#         return "금요일"
#     elif weekday == 5:
#         return "토요일"
#     elif weekday == 6:
#         return "일요일"
    
    return weekday

In [136]:
def make_hour(col): # 시간대별
    col = str(col)
    hour = int(col[8:10])
#     hour = str(hour) + "시"
    return hour

In [137]:
train_prob['weekday'] = train_prob.time.apply(lambda x: make_weekday(x))
train_prob['hour'] = train_prob.time.apply(lambda x: make_hour(x))

* 요일별 불만 제기 확인 결과 : <b>월요일 → 수요일 → 금요일 → 화요일 → 목요일 → 토요일 → 일요일</b> 순으로 많음을 알 수 있다

In [138]:
train_prob['weekday'].value_counts() # 요일별 불만제기

0    1332
2     868
4     853
1     809
3     809
5     431
6     327
Name: weekday, dtype: int64

* 시간대별 불만 제거 확인 결과 : <b>점심 → 저녁 → 오후 → 밤 → 새벽 → 아침</b> 순으로 많음을 알 수 있다

In [139]:
train_prob['hour'].value_counts() # 시간별 불만 제기

12    627
11    600
13    518
19    507
18    492
15    489
16    460
17    432
20    233
22    233
14    228
21    204
23    175
0     108
1      67
2      15
10     14
9       8
5       5
8       5
3       4
4       3
6       1
7       1
Name: hour, dtype: int64

* 평일/주말 불만 제기 확인 결과 : <b>평일이 주말보다</b> 불만 제기가 많음을 알 수 있다

In [140]:
weekend = train_prob[(train_prob['weekday'] == 5) | (train_prob['weekday'] == 6)]
weekdays = train_prob[(train_prob['weekday'] != 5) & (train_prob['weekday'] != 6)]

In [141]:
week_ratio = len(weekend) / len(train_prob) #주말 제기 확률
print("평일에 불만을 제기할 확률 : {0}%\n주말에 불만을 제기할 확률 : {1}%".
      format(round((1-week_ratio)*100, 3), (week_ratio)*100, 3))

평일에 불만을 제기할 확률 : 86.038%
주말에 불만을 제기할 확률 : 13.962055627187327%


In [142]:
weekend_open = weekend[(weekend['weekday'] != 6) &(weekend['hour'] >= 9) & (weekend['hour'] <= 13)]
weekdays_open = weekdays[(weekdays['hour'] >= 9) & (weekdays['hour'] <= 18)]
print("운영시간 중 불만 제기 확률 : {0}%".format(round((len(weekdays_open)+ len(weekend_open)) / len(train_prob) * 100, 3)))

운영시간 중 불만 제기 확률 : 64.892%


In [143]:
# 다시 Datetime으로 Type이 변경된 데이터 로드
train_err = pd.read_csv(save_path+"train_err_data_time.csv")
train_qual = pd.read_csv(save_path+"train_quality_data_time.csv")
train_prob = pd.read_csv(save_path+"train_problem_data_time.csv")

test_err = pd.read_csv(save_path+"test_err_data_time.csv")
test_qual = pd.read_csv(save_path+"test_quality_data_time.csv")

---

# 5. Err Data 해석

### 에러가 많이 발생할수록, 불만 제기 확률이 높을 것이라는 가설을 세우고 → 이를 검정하기 위한 `Errtype Data` 해석

---

## 5-1. Errtype

###### 1. Errtype 종류에 상관없이 Errtype의 수를 Count

In [144]:
err_total = train_err.copy()

id_error = err_total[['user_id','errtype']].values
error = np.zeros(15000)

In [145]:
for person_idx, err,  in tqdm(id_error):
    error[person_idx-10000] += 1

100%|██████████| 16554663/16554663 [00:37<00:00, 444848.74it/s]


In [146]:
problem_user = sorted(list(train_prob.user_id.unique()))
prob_user = [i-10000 for i in problem_user]

In [147]:
psum = [] # 불만 제기한 사람들 각각의 Total Errtype Count
nsum = [] # 불만 제기하지않은 사람들 각각의 Total Errtype Count

for i in range(len(error)):
    if i in prob_user:
        psum.append(error[i])
    else:
        nsum.append(error[i])
        
print("불만 제기한 사람들의 Total Errtype Count의 평균은 {0}입니다.".format(sum(psum)/len(psum)))
print("불만 제기하지않은 사람들의 Total Errtype Count의 평균은 {0}입니다.".format(sum(nsum)/len(nsum)))

불만 제기한 사람들의 Total Errtype Count의 평균은 1517.7562입니다.
불만 제기하지않은 사람들의 Total Errtype Count의 평균은 896.5882입니다.


* Total Errtype을 많이 가지고 있는 사람들이 불만 제기를 많이 한다는 사실을 알 수 있었다.  

→ &nbsp;따라서 Errtype을 Count하는 것이 유의미하다는 결론을 내리고, Errtype의 이동평균, 이동최대값, 이동표준편차을 살펴볼 필요가 있다.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:gray">(시간대별 이동평균, 하루 이동평균, 3일 이동평균, 7일 이동평균 + 시간대별 이동최대값, 하루 이동최대값, 3일 이동최대값, 7일 이동최대값 등)</span>

<center> <b>예시 (Count)</b></center>  

|User ID|Errtype의 3일 이동평균|Errtype의 3일 이동최대값|Errtype의 3일 이동표준편차|  
|---|---|---|---|
|User 1|7.5|322.0|0.9142|  
|User 2|2.3|2423.0|141.93|
|User 3|0.0|11.0|0.029|
|User 4|1.7|299.0|0.0113|
|User 5|3.0|4.0|1.21|

<br>
<center><span style="color:gray">(엄밀하게는 이동평균의 평균, 이동최대값의 평균, 이동표준편차의 평균을 봄)</span></center>

###### 2. Errtype이 불만 제기에 중요한 변수임을 알았다. 그렇다면 <u><b>Errtype의 종류마다 영향력이 다를까?</b></u>  

In [191]:
type5_id = sorted(list(train_err[train_err['errtype'] == 40].user_id.unique()))
print("Errtype 18을 가진 유저들의 수 : {}명".format(len(type5_id)))
print("Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : {}명".format(len(train_prob[train_prob['user_id'].isin(type5_id)].user_id.unique())))
print("비율 : %.3f%%"%(len(train_prob[train_prob['user_id'].isin(type5_id)].user_id.unique())/len(type5_id) * 100))

Errtype 18을 가진 유저들의 수 : 10462명
Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : 4052명
비율 : 38.731%


In [149]:
type18_id = sorted(list(train_err[train_err['errtype'] == 18].user_id.unique()))
print("Errtype 18을 가진 유저들의 수 : {}명".format(len(type18_id)))
print("Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : {}명".format(len(train_prob[train_prob['user_id'].isin(type18_id)].user_id.unique())))
print("비율 : %.3f%%"%(len(train_prob[train_prob['user_id'].isin(type18_id)].user_id.unique())/len(type18_id) * 100))

Errtype 18을 가진 유저들의 수 : 1768명
Errtype 18을 가진 유저중에서 불만을 제기한 사람의 수 : 1521명
비율 : 86.029%


→ &nbsp;특정 Errtype은 불만 제기에 큰 영향을 미침을 알 수 있다.

→ &nbsp;따라서, Errtype을 종류별로 Count하는 파생변수를 살펴볼 필요가 있다<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:gray">(종류에 따라서, 동일하게 시간대별 이동평균, 하루 이동평균, 3일 이동평균, 7일 이동평균 + 시간대별 이동최대값, 하루 이동최대값 ˙˙˙ 등)</span>

<center> <b>예시 (Count)</b></center>  

|User ID|Errtype18의 3일 이동평균|Errtype18의 3일 이동최대값|Errtype18의 3일 이동표준편차|  
|---|---|---|---|
|User 1|1.5|3.0|0.142|  
|User 2|1.0|22.0|14.93|
|User 3|0.0|1.0|0.09|
|User 4|1.5|2.0|0.013|
|User 5|3.0|5.0|1.2|

<br>
<center><span style="color:gray">(엄밀하게는 이동평균의 평균, 이동최대값의 평균, 이동표준편차의 평균을 봄)</span></center>

###### 3. Errtype과 Fwver & Model과의 관계

In [150]:
# 모델 버전마다 Errtype이 다름을 보여주는 코드
for i in range(9):
    nm = 'model_' + str(i)
    print("모델 버전", nm, "→", sorted(list(train_err[train_err['model_nm'] == nm]['errtype'].unique()))[:15])

모델 버전 model_0 → [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_1 → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
모델 버전 model_2 → [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_3 → [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
모델 버전 model_4 → [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
모델 버전 model_5 → [2, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
모델 버전 model_6 → [5]
모델 버전 model_7 → [3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18]
모델 버전 model_8 → [2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [151]:
# 펌웨어 버전마다 Errtype이 다름을 보여주는 코드
so_fw = sorted(list(train_err.fwver.unique()))

for i in so_fw:
    nm = i
    print("펌웨어 버전 →", i,sorted(list(train_err[train_err['fwver'] == nm]['errtype'].unique()))[:20])

펌웨어 버전 → 03.11.1141 [1, 5, 6, 7, 14, 27, 28, 30]
펌웨어 버전 → 03.11.1149 [1, 4, 5, 6, 7, 12, 14, 27, 28]
펌웨어 버전 → 03.11.1167 [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 27, 28, 30]
펌웨어 버전 → 04.16.2641 [3, 4, 6, 7, 11, 12, 13, 14, 15, 16, 22, 23, 26]
펌웨어 버전 → 04.16.3345 [3, 4, 10, 11, 12, 14, 15, 16, 26]
펌웨어 버전 → 04.16.3439 [6, 7, 11, 12, 14, 15, 16, 22, 23, 26, 27, 28, 31, 33, 34, 35, 38, 40]
펌웨어 버전 → 04.16.3553 [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
펌웨어 버전 → 04.16.3569 [5, 10, 11, 12, 15, 16, 17, 22, 23, 26, 31, 32, 33, 34, 42]
펌웨어 버전 → 04.16.3571 [2, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
펌웨어 버전 → 04.22.1442 [4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 23, 25, 26]
펌웨어 버전 → 04.22.1656 [4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 22, 23, 26, 27, 28]
펌웨어 버전 → 04.22.1666 [12, 13, 14, 38]
펌웨어 버전 → 04.22.1684 [1, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 25, 26, 27]
펌웨어 버전 → 04.22.1750 [1, 2, 4, 

→ &nbsp;Errtype이 모델 버전 또는 펌웨어 버전에 종속되지 않음을 알 수 있다.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:gray">단, 모델6에는 펌웨어 버전 '10'과 '8.5.3'이 있고 Errtype은 5만 존재한다.</span>

---

## 5-2. Errcode

###### 1. Errtype과 비슷하게 Errcode도 Error의 정보를 알려주는 Feature이다. 먼저 <u>둘의 관계를 살펴보자</u>

In [152]:
print(sorted(list(train_err.errcode.unique())[:50], reverse = True))
print("\nErrcode의 종류 : {}개".format(len(list(train_err.errcode.unique()))))

['terminate by peer user', 'standby', 'connectionterminated by local host', 'connection timeout', 'connection fail to establish', 'connection fail for LMP response timout', 'active', 'V-21008', 'UNKNOWN', 'U-81009', 'S-65002', 'S-64002', 'S-61001', 'Q-64002', 'NFANDROID2', 'J-30021', 'B-A8002', '95', '94', '93', '91', '90', '89', '88', '87', '86', '85', '84', '83', '82', '81', '80', '8.0', '79', '78', '77', '700001', '6796', '6467', '6', '5738', '5507', '4893', '4', '3', '2', '14', '13', '1', '0']

Errcode의 종류 : 2805개


In [153]:
ey_ec = train_err.groupby('errtype')['errcode'].unique().to_frame()
display(ey_ec)

,errcode
errtype,
1,"[0, P-44010, P-41011, P-41007 , P-44010 , P-41..."
2,"[1, 0]"
3,"[1, 2, 0]"
4,"[0, 1]"
5,"[B-A8002, Q-64002, S-61001, U-81009, V-21008, ..."
6,"[1, 14]"
7,"[1, 14]"
8,"[PHONE_ERR, PUBLIC_ERR, 20]"
9,"[V-21002, V-21005, 1, C-14014, V-21008, C-1203..."


→ &nbsp; <b>Errtype은 Errcode를 군집화시킨 변수</b>임을 알 수 있다.

###### 2. Errtype은 중요한 변수였고. Errcode가 Errtype에 속한다. 그렇다면  <u><b>Errcode도 종류마다 불만 제기에 미치는 영향력이 다를까?</b></u>  

* 먼저, Errcode의 대표적인 종류는 다음과 같다.
    - P-41001과 같은 <u>`문자+숫자형`</u> 에러코드
    - 1, 13, 14와 같은 <u>`두자리수 이하`</u>의 에러코드
    - 3113, 3395와 같은 <u>`세자리수 이상`</u>의 에러코드
    - Connection Timeout, L2CAP Connection Cancelled와 같은 <u>`문장형`</u> 에러코드

In [154]:
ey_ec = ey_ec.reset_index()
display(sorted(list(ey_ec[ey_ec['errtype'] == 1].errcode.values[0][2:8])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 5].errcode.values[0][6:12])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 14].errcode.values[0][:])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 25].errcode.values[0][3:9])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 32].errcode.values[0][3:9])))
display(sorted(list(ey_ec[ey_ec['errtype'] == 38].errcode.values[0][3:9])))

['P-41001', 'P-41007', 'P-41007 ', 'P-41011', 'P-41011 ', 'P-44010 ']

['C-11017', 'H-51042', 'J-30021', 'Q-64001', 'S-64002', 'S-65002']

['1', '13', '14']

['L2CAP connection cancelled',
 'UNKNOWN',
 'connection fail to establish',
 'connection timeout',
 'connectionterminated by local host',
 'terminate by peer user']

['77', '78', '84', '85', '86', '90']

['3113', '3395', '3674', '39391', '4893', '5507']

→ &nbsp;이 중에서 <b>불만 제기 확률이 높은 Errcode</b>에는 어떤 것들이 있을까?

In [155]:
problem = train_prob.copy()
p_id = sorted(problem['user_id'].unique())

errin = err_total[err_total['user_id'].isin(p_id)]['errcode'].value_counts().to_dict()
errout = err_total[~err_total['user_id'].isin(p_id)]['errcode'].value_counts().to_dict()

In [156]:
errcode_prob = []
most_error = []

for i in errin:
    try :
        errcode_prob.append((i,errin[i]/(errin[i]+errout[i]),errin[i],errout[i]))
    except :
        errcode_prob.append((i,1,errin[i],0))
    
for i in errout:
    if i in errin:
        continue
    errcode_prob.append((i,0,0,errout[i]))

In [157]:
errcode_prob.sort(key = lambda x: x[1], reverse = True)

In [158]:
most_error = []
most_not_error = []

for i in errcode_prob:
    if i[2] >= 20 and i[3] >= 20:
        if i[1] >= 0.75:
            most_error.append(i[0])
        elif i[1] <= 0.25 :
            most_not_error.append(i[0])

print("불만 제기 확률이 높은 에러코드 : {0}".format(most_error))
print("불만 제기 확률이 낮은 에러코드 : {0}".format(most_not_error))

불만 제기 확률이 높은 에러코드 : ['scanning timeout', '5', '6', 'V-21008', 'terminate by peer user', 'V-21005']
불만 제기 확률이 낮은 에러코드 : ['Q-64001', 'Q-64002', 'P-44010', 'PHONE_ERR', 'B-51049', 'H-51049']


<b>특정 에러 코드가 불만 제기에 유의미한 영향을 미침</b>임을 알 수 있다.<br>
→ &nbsp; 따라서 <span style="color:red">에러 코드와 불만 제기 간의 관계</span>를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:gray">(이것 또한 이동평균을 이용하여 Errtype과 같은 형식으로 만든다)</span>

---

## 5-3. Fwver (펌웨어 버전)

* 먼저, Fwver의 대표적인 종류는 다음과 같다.
    - '03'으로 시작하는 펌웨어 버전
    - '04'으로 시작하는 펌웨어 버전
    - '05'으로 시작하는 펌웨어 버전
    - '10', '8.5.3' 특수한 펌웨어 버전

In [159]:
print(sorted(train_err.fwver.unique()))

['03.11.1141', '03.11.1149', '03.11.1167', '04.16.2641', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571', '04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778', '04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2571', '04.82.1684', '04.82.1730', '04.82.1778', '05.15.2090', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104', '05.66.3237', '05.66.3571', '10', '8.5.3']


###### 1. 펌웨어 버전에 따라서, 불만 제기 확률이 달라질까?

In [160]:
def relation_fw_complain(data): # 펌웨어 버젼과 불만 제기와의 관계
    fwlst = list(data['fwver'].value_counts().keys())
    result = []
    for i in tqdm(fwlst):
        f1 = sorted(list(data[data['fwver'] == i].user_id.unique()))
        f2 = sorted(list(train_prob[train_prob['user_id'].isin(f1)]['user_id'].unique()))
        if (len(f2)/len(f1) >= 0.5) and (len(f1) >= 5):
            result.append([i, len(f2)/len(f1), f2, f1])
    return result

In [161]:
def view_relation_fw_complain(data):
    result = relation_fw_complain(data)
    for i in range(len(result)):
        fw = result[i][0]
        fp = result[i][1] * 100
        fww = result[i][2]
        fwp = result[i][3]
        print("펌웨어 버전 %s를 사용중인 사람들 가운데, %.2f%%는 불만을 제기하였습니다. \n불만 제기를 한 사람의 수는 전체 %s명중 %s명입니다.\n"%(fw, fp, len(fwp), len(fww)))

In [162]:
view_relation_fw_complain(train_err)

100%|██████████| 37/37 [00:42<00:00,  1.16s/it]

펌웨어 버전 04.16.3571를 사용중인 사람들 가운데, 58.76%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 502명중 295명입니다.

펌웨어 버전 05.66.3237를 사용중인 사람들 가운데, 52.46%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 61명중 32명입니다.

펌웨어 버전 04.82.1778를 사용중인 사람들 가운데, 50.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 40명중 20명입니다.

펌웨어 버전 05.66.3571를 사용중인 사람들 가운데, 53.33%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 15명중 8명입니다.

펌웨어 버전 04.33.1149를 사용중인 사람들 가운데, 70.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 160명중 112명입니다.

펌웨어 버전 04.73.2571를 사용중인 사람들 가운데, 66.67%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 12명중 8명입니다.

펌웨어 버전 04.33.1125를 사용중인 사람들 가운데, 66.67%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 6명중 4명입니다.



<b>특정 펌웨어 버전이 불만 제기에 유의미한 영향을 미침</b>임을 알 수 있다.<br>
→ &nbsp; 따라서 <span style="color:red">펌웨어 버전과 불만 제기 간의 관계</span>를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.

---

In [163]:
view_relation_fw_complain(train_qual)

100%|██████████| 27/27 [00:01<00:00, 14.95it/s]

펌웨어 버전 04.22.1684를 사용중인 사람들 가운데, 59.09%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 13명입니다.

펌웨어 버전 09.17.1431를 사용중인 사람들 가운데, 96.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 25명중 24명입니다.

펌웨어 버전 04.16.3571를 사용중인 사람들 가운데, 63.24%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 68명중 43명입니다.

펌웨어 버전 04.82.1684를 사용중인 사람들 가운데, 54.55%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 12명입니다.

펌웨어 버전 04.33.1149를 사용중인 사람들 가운데, 70.27%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 37명중 26명입니다.

펌웨어 버전 05.66.3237를 사용중인 사람들 가운데, 75.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 8명중 6명입니다.

펌웨어 버전 04.82.1778를 사용중인 사람들 가운데, 61.54%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 13명중 8명입니다.

펌웨어 버전 05.66.3571를 사용중인 사람들 가운데, 60.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 5명중 3명입니다.



<center><span style="color:gray">(참고 : Train Quality Data의 Fwver의 경우 : 같은 함수를 적용, 하단에 기재)</span></center>

---

###### 2. Fwver 앞 4자리만 같고, 뒤 4자리가 다른 버전들이 존재 → <u>펌웨어 버전 업데이트</u>가 유의미할까?

In [164]:
train_fw = train_err.copy()
train_fw['fwver'] = train_fw['fwver'].apply(lambda x : x.replace(".",""))
train_fw['fwver'] = train_fw['fwver'].apply(lambda x : int(x))

In [165]:
te_uf = train_fw[['user_id','fwver']]
fw_df = ~(te_uf == te_uf.shift(1))

In [166]:
logical = (fw_df.user_id.apply(int) + fw_df.fwver.apply(int)) > 0
tre_fw_counts = te_uf[logical]

In [167]:
user_id_2_fw = tre_fw_counts.user_id.value_counts()
user_id_lst_2_fw = list(user_id_2_fw.loc[user_id_2_fw == 2].to_frame().index)

In [168]:
sort_two_fw_user = train_fw.loc[train_fw.user_id.isin(user_id_lst_2_fw)]. \
drop_duplicates(['user_id','fwver'],keep='last'). \
drop_duplicates('user_id', keep='first').sort_values("time").user_id

In [169]:
operator1 = len(user_id_lst_2_fw)
operator2 = len(list(train_prob[train_prob['user_id'].isin(user_id_lst_2_fw)].user_id.unique()))

print("펌웨어 버전이 변하는 시기에 불만을 제기한 사람 {0}%".format(round(operator2 / operator1 * 100,2)))

펌웨어 버전이 변하는 시기에 불만을 제기한 사람 36.45%


<b>펌웨어 버전 업데이트가 불만 제기에 유의미한 영향을 미침</b>임을 알 수 있다.<br>
 → &nbsp; 따라서 <span style="color:red">펌웨어 버전의 변화와 불만 제기 간이 관계</span>를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.

---

## 5-4. Model (모델 버전)

* 먼저, Model의 종류는 다음과 같다.
    - Model_0부터 Model_8까지 총 `9가지 종류`의 모델 넘버가 존재

In [170]:
print(sorted(list(train_err.model_nm.unique())))

['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6', 'model_7', 'model_8']


###### 1. 모델 버전과 펌웨어 버전의 관계

In [171]:
display(train_err.groupby('model_nm')['fwver'].unique().to_frame().reset_index())

,model_nm,fwver
0,model_0,"[04.22.1750, 04.22.1778, 04.22.1684, 04.22.166..."
1,model_1,"[04.16.3553, 04.16.3571, 04.16.3439, 04.16.356..."
2,model_2,"[04.33.1185, 04.33.1261, 04.33.1149, 04.33.117..."
3,model_3,"[05.15.2138, 05.15.2120, 05.15.2090, 05.15.310..."
4,model_4,"[03.11.1149, 03.11.1167, 03.11.1141]"
5,model_5,"[04.82.1684, 04.82.1778, 04.82.1730]"
6,model_6,"[10, 8.5.3]"
7,model_7,"[05.66.3237, 05.66.3571]"
8,model_8,"[04.73.2237, 04.73.2571]"


In [172]:
# 모델마다 펌웨어 버전이 다름을 보여주는 코드
for i in range(9):
    nm = 'model_' + str(i)
    print("모델 : ", nm, "→\n", sorted(list(train_err[train_err['model_nm'] == nm]['fwver'].unique())))

모델 :  model_0 →
 ['04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778']
모델 :  model_1 →
 ['04.16.2641', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571']
모델 :  model_2 →
 ['04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261']
모델 :  model_3 →
 ['05.15.2090', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104']
모델 :  model_4 →
 ['03.11.1141', '03.11.1149', '03.11.1167']
모델 :  model_5 →
 ['04.82.1684', '04.82.1730', '04.82.1778']
모델 :  model_6 →
 ['10', '8.5.3']
모델 :  model_7 →
 ['05.66.3237', '05.66.3571']
모델 :  model_8 →
 ['04.73.2237', '04.73.2571']


 → &nbsp; 모델과 펌웨어 버전은 종속적인 관계로, <u><b>모델별 펌웨어 버전이 존재함</b></u>을 알 수 있다.

###### 2. <u>모델 버전 변경</u>이 유의미할까?

In [173]:
train_md = train_err.copy()
train_md['model_nm'] = train_md['model_nm'].apply(lambda x : x.replace("_",""))
train_md['model_nm'] = train_md['model_nm'].apply(lambda x : str(x)[5])

In [174]:
te_uf = train_md[['user_id','model_nm']]
md_df = ~(te_uf == te_uf.shift(1))

In [175]:
logical = (md_df.user_id.apply(int) + md_df.model_nm.apply(int)) > 0
tre_md_counts = te_uf[logical]

In [176]:
user_id_2_md = tre_md_counts.user_id.value_counts()
user_id_lst_2_md = list(user_id_2_md.loc[user_id_2_md == 2].to_frame().index)

In [177]:
sort_two_md_user = train_md.loc[train_md.user_id.isin(user_id_lst_2_md)]. \
drop_duplicates(['user_id','model_nm'],keep='last'). \
drop_duplicates('user_id', keep='first').sort_values("time").user_id

In [178]:
operator1_m = len(user_id_lst_2_md)
operator2_m = len(list(train_prob[train_prob['user_id'].isin(user_id_lst_2_md)].user_id.unique()))

print("모델이 변하는 시기에 불만을 제기한 사람 {0}%".format(round(operator2_m / operator1_m * 100,2)))

모델이 변하는 시기에 불만을 제기한 사람 91.57%


<b>모델 버전 변경이 불만 제기에 아주 유의미한 영향을 미침</b>임을 알 수 있다.<br>
 → &nbsp; 따라서 <span style="color:red">모델 버전의 변화와 불만 제기 간의 관계</span>를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.<br>
 → &nbsp; <span style="color:gray">이는 모델별 펌웨어 버전이 존재하기 때문에, 펌웨어 버전의 변경만으로 대신 증명할 수 있다.</span>

---

# 6. Quality Data 수치 분석

---

### 정의 : 사용자의 시스템 작동 중 문제가 발생하면 `측정 가능한 지표`들로 해당 시점으로부터 2시간 단위 수집

In [179]:
train_qual.iloc[:,[1,0,3,4,5,6,7,8,9,10,11,12,13,14,15]][12372:12390]

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
12372,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12373,10265,2020-11-08 07:50:00,0,0,1,0,0,4,1,13,1,119,132,1,2
12374,10265,2020-11-08 07:50:00,0,0,4,0,0,12,0,13,1,119,132,1,2
12375,10265,2020-11-08 07:50:00,0,1,21,0,0,23,12,13,1,119,132,0,2
12376,10265,2020-11-08 07:50:00,0,0,54,0,0,54,0,13,1,119,132,0,2
12377,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12378,10265,2020-11-08 07:50:00,0,0,37,0,0,37,0,13,1,119,132,0,2
12379,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12380,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2
12381,10265,2020-11-08 07:50:00,0,0,0,0,0,0,0,13,1,119,132,0,2


In [180]:
train_qual.iloc[:,[1,0,3,4,5,6,7,8,9,10,11,12,13,14,15]].describe()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000
mean,17583.75951,3.59968,-0.16982,4.75090,0.00000,0.00000,78.14516,2.11059,27.52441,0.15948,59.20809,939.93923,-0.17910,0.04810
std,4400.24714,446.47710,0.69788,586.24020,0.00000,0.00000,2335.76090,33.45625,325.64862,5.27364,3362.65838,16934.49764,0.39630,0.30941
min,10000.00000,-1.00000,-1.00000,-1.00000,0.00000,0.00000,-1.00000,-1.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
25%,13695.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.00000,0.00000,0.00000
50%,17507.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,6.00000,0.00000,0.00000
75%,21532.50000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,41.00000,0.00000,0.00000
max,24997.00000,157667.00000,171.00000,191859.00000,0.00000,0.00000,637385.00000,600.00000,7200.00000,1317.00000,397424.00000,1910175.00000,14.00000,14.00000


---

## 6-1. Quality 값

###### 1. Quality는 측정가능한 지표로 만들어진 값이다. 그러나 해당 값들의 범위를 살펴보면 [0, ∞)가 아닌 [-1, ∞)이다.

### "<b>`-1`</b>값은 측정이 되지 못한, <b>`알 수 없음(또는 문제 있음)`</b>을 나타내는 지표일 것이다" 라는 <span style="color:red"><u>가설</u></span>을 세움

&nbsp;&nbsp;&nbsp; → &nbsp;-1값을 많이 갖고 있는 유저가 불만제기를 더 많이 하였는지 확인

In [185]:
train_qual[train_qual['quality_0'] == -1].iloc[:,[1,0,2,3,4,5,8,9,14,6,7,10,11,12,13]][:15]

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_5,quality_6,quality_11,quality_3,quality_4,quality_7,quality_8,quality_9,quality_10
72,10002,2020-11-18 00:10:00,05.15.2138,-1,-1,-1,-1,-1,-1,0,0,0,0,0,3
73,10002,2020-11-18 00:10:00,05.15.2138,-1,-1,-1,-1,-1,-1,0,0,0,0,0,3
130,10004,2020-11-02 23:20:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,0,0,0,3
131,10004,2020-11-02 23:20:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,0,0,0,3
158,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4
159,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4
160,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4
161,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4
162,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4
163,10005,2020-11-19 04:00:00,04.22.1750,-1,-1,-1,-1,-1,-1,0,0,36,0,0,4


In [103]:
# -1을 갖고 있는 유저의 수
len(sorted(list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())))

5567

In [104]:
# -1을 많이 가지고 있는 유저 상위 100명
minus_id_top100 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())[:100]

In [105]:
# 상위 100명중, 불만을 제기한 사람의 수 : 70명 (70.0%)
len(train_prob[train_prob['user_id'].isin(minus_id_top100)].user_id.unique())

70

In [106]:
# -1을 많이 가지고 있는 유저 상위 900명
minus_id_top900 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())[100:1000]

In [107]:
# 상위 900명중, 불만을 제기한 사람의 수 : 505명 (56.1%)
len(train_prob[train_prob['user_id'].isin(minus_id_top900)].user_id.unique())

505

In [108]:
# -1을 많이 가지고 있는 유저 상위 2000명
minus_id_top2000 = list(train_qual[train_qual['quality_0'] == -1].groupby('user_id')['quality_0'].count().to_frame(). \
reset_index().sort_values(ascending=False, by='quality_0').user_id.unique())[1000:3000]

In [109]:
# 상위 2000명중, 불만을 제기한 사람의 수 : 799명 (39.9%)
len(train_prob[train_prob['user_id'].isin(minus_id_top2000)].user_id.unique())

799

"-1을 더 많이 가지고 있을수록 <b>불만을 더 많이 제기함</b>"을 알 수 있다.<br>
 → &nbsp; 따라서 <span style="color:red">-1은 불만 제기와 관련이 있는 수치이기 때문에</span> 이를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.<br>
 → &nbsp; <span style="color:gray">User별 Quality_0부터 Quality_12까지 -1의 갯수를 카운팅하는 파생변수를 만들었다.</span>

<center> <b>예시 </b></center>  

|User ID|Quality_0_minus_count|Quality_1_minus_count|˙˙˙|  
|-------|-------|-------|-------|
|User 1|0|0|˙˙˙|  
|User 2|9|9|˙˙˙|  
|User 3|15|14|˙˙˙|  
|User 4|0|0|˙˙˙|  

<br>
<center><span style="color:gray">(각 유저별로 Quality -1값의 개수를 카운트한 파생변수)</span></center>

###### 2. 마이너스 1값을 카운팅 한 뒤, 해당 값들을 제외한 나머지 변수들의 수치를 해석하기 위하여 `-1`을 `0`으로 변경

In [110]:
for i in train_qual.columns[3:]:
    idx = train_qual[train_qual[i] == -1].index
    train_qual.loc[idx,i] = 0

###### 3. Quality Data 기본 수치 해석

In [111]:
train_qual.iloc[:,3:].describe()

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000,788544.00000
mean,3.76559,0.01329,4.93401,0.00000,0.00000,78.32827,2.29370,27.52441,0.15948,59.20809,939.93923,0.00401,0.04810
std,446.47555,0.57670,586.23853,0.00000,0.00000,2335.75473,33.44146,325.64862,5.27364,3362.65838,16934.49764,0.07747,0.30941
min,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.00000,0.00000,0.00000
50%,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,6.00000,0.00000,0.00000
75%,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,41.00000,0.00000,0.00000
max,157667.00000,171.00000,191859.00000,0.00000,0.00000,637385.00000,600.00000,7200.00000,1317.00000,397424.00000,1910175.00000,14.00000,14.00000


→ &nbsp; Quality_3과 Quality_4는 모두 0값만을 가지고 있음을 확인<br>
→ &nbsp; Quality_0, Quality_2, Quality_5, Quality_9, Quality_10의 <u>`분산과 최댓값`</u>이 크다는 것을 확인 : 불만 제기와의 관계 확인

In [112]:
# Quality_2, 5, 9, 10과 불만 제기와의 관계 확인
for i in [0, 2, 5, 9, 10]:
    tq = train_qual.groupby('user_id')['quality_'+str(i)].max().to_frame().reset_index().sort_values(ascending=False, by='quality_'+str(i))
    tq_id = list(tq.user_id.values)[100:900]
    tp_id = train_prob[train_prob['user_id'].isin(tq_id)].user_id.unique()
    print("퀄리티 {0}을 합한 값이 큰 상위 유저 900명 중 {1}명이 불만을 제기하였습니다.  {2}%".format(i, len(tp_id), len(tp_id)/len(tq_id)*100))

퀄리티 0을 합한 값이 큰 상위 유저 900명 중 290명이 불만을 제기하였습니다.  36.25%
퀄리티 2을 합한 값이 큰 상위 유저 900명 중 284명이 불만을 제기하였습니다.  35.5%
퀄리티 5을 합한 값이 큰 상위 유저 900명 중 375명이 불만을 제기하였습니다.  46.875%
퀄리티 9을 합한 값이 큰 상위 유저 900명 중 282명이 불만을 제기하였습니다.  35.25%
퀄리티 10을 합한 값이 큰 상위 유저 900명 중 401명이 불만을 제기하였습니다.  50.125%


In [113]:
# Quality_2, 5, 9, 10과 불만 제기와의 관계 확인
for i in [0, 2, 5, 9, 10]:
    tq = train_qual.groupby('user_id')['quality_'+str(i)].max().to_frame().reset_index().sort_values(ascending=False, by='quality_'+str(i))
    tq_id = list(tq.user_id.values)[:100]
    tp_id = train_prob[train_prob['user_id'].isin(tq_id)].user_id.unique()
    print("퀄리티 {0}을 합한 값이 큰 상위 유저 100명 중 {1}명이 불만을 제기하였습니다.  {2}%".format(i, len(tp_id), len(tp_id)/len(tq_id)*100))

퀄리티 0을 합한 값이 큰 상위 유저 100명 중 36명이 불만을 제기하였습니다.  36.0%
퀄리티 2을 합한 값이 큰 상위 유저 100명 중 39명이 불만을 제기하였습니다.  39.0%
퀄리티 5을 합한 값이 큰 상위 유저 100명 중 74명이 불만을 제기하였습니다.  74.0%
퀄리티 9을 합한 값이 큰 상위 유저 100명 중 39명이 불만을 제기하였습니다.  39.0%
퀄리티 10을 합한 값이 큰 상위 유저 100명 중 74명이 불만을 제기하였습니다.  74.0%


#### → &nbsp; 퀄리티 합이 높을수록, 불만 제기 확률이 높아짐을 알 수 있음

###### 4. 나머지 Quality 변수들은 0부터 무한대 사이의 값을 가지고 있으므로, 이것이 시스템 품질을 측정한 하나의 값으로 가정
 → &nbsp; Quality 변수들의 간의 관계를 해석하기 위하여 `Correlation`을 이용  
 → &nbsp; 유저별 Quality값들의 합이 높은 것과 불만 제기의 관계 해석

In [114]:
qual_sum = pd.DataFrame({'user_id' : np.arange(10000,25000)})

for i in train_qual.columns[3:]:
    qq = train_qual.groupby('user_id')[i].sum().to_frame().reset_index()
    qual_sum = pd.merge(qual_sum, qq, how='left', on ='user_id')
    qual_sum = qual_sum.fillna(0)

In [115]:
qual_sum.head()

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.00000,0.00000,0.00000,0.00000,0.00000,12.00000,0.00000,0.00000,0.00000,0.00000,144.00000,0.00000,0.00000
1,10001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,10002,2.00000,0.00000,1.00000,0.00000,0.00000,31.00000,46.00000,552.00000,0.00000,12.00000,372.00000,0.00000,0.00000
3,10003,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,10004,0.00000,0.00000,0.00000,0.00000,0.00000,4.00000,87.00000,1044.00000,0.00000,0.00000,48.00000,0.00000,0.00000


In [194]:
# del qual_sum['quality_3']
# del qual_sum['quality_4']

In [195]:
qual_sum.corr()

,user_id,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,1.00000,0.01775,0.00618,0.01177,-0.00422,-0.00978,-0.00978,0.00618,0.01177,-0.00422,0.00959,0.00959
quality_0,0.01775,1.00000,-0.00106,0.88105,0.04295,0.00066,0.00066,-0.00106,0.88105,0.04295,0.00106,0.00106
quality_1,0.00618,-0.00106,1.00000,-0.00035,0.02997,0.00733,0.00733,1.00000,-0.00035,0.02997,0.10275,0.10275
quality_2,0.01177,0.88105,-0.00035,1.00000,0.04433,0.00107,0.00107,-0.00035,1.00000,0.04433,0.02326,0.02326
quality_5,-0.00422,0.04295,0.02997,0.04433,1.00000,0.01970,0.01970,0.02997,0.04433,1.00000,0.07244,0.07244
quality_6,-0.00978,0.00066,0.00733,0.00107,0.01970,1.00000,1.00000,0.00733,0.00107,0.01970,0.00061,0.00061
quality_7,-0.00978,0.00066,0.00733,0.00107,0.01970,1.00000,1.00000,0.00733,0.00107,0.01970,0.00061,0.00061
quality_8,0.00618,-0.00106,1.00000,-0.00035,0.02997,0.00733,0.00733,1.00000,-0.00035,0.02997,0.10275,0.10275
quality_9,0.01177,0.88105,-0.00035,1.00000,0.04433,0.00107,0.00107,-0.00035,1.00000,0.04433,0.02326,0.02326
quality_10,-0.00422,0.04295,0.02997,0.04433,1.00000,0.01970,0.01970,0.02997,0.04433,1.00000,0.07244,0.07244


### → `Quality` 값을 유저별로 합하여 만든 파생 변수들 중에, <span style="color:red"><u>완벽한 선형관계</u></span>를 보이는 관계를 발견


|완벽한 선형관계|상관계수|  
|-------|-------|  
|Quality_1 ~ Quality_8|1.000|  
|Quality_2 ~ Quality_9|1.000|  
|Quality_5 ~ Quality_10|1.000|  
|Quality_6 ~ Quality_7|1.000|  
|Quality_11 ~ Quality_12|1.000|<br><br>

<center><span style="color : gray">* 실제 데이터를 살펴본 결과, <b>$y=12x$</b> 관계가 있음을 확인<br>
&nbsp;&nbsp;Example) Quality_8 = Quality_1 * 4</span></center>

In [117]:
display(qual_sum.iloc[:,[2,9]][2505:2510])
display(qual_sum.iloc[:,[3,10]][250:255])
display(qual_sum.iloc[:,[6,11]][2505:2510])
display(qual_sum.iloc[:,[7,8]][2505:2510])
display(qual_sum.iloc[:,[12,13]][150:155])

,quality_1,quality_8
2505,4.00000,48.00000
2506,0.00000,0.00000
2507,0.00000,0.00000
2508,2.00000,24.00000
2509,0.00000,0.00000


,quality_2,quality_9
250,0.00000,0.00000
251,70.00000,840.00000
252,0.00000,0.00000
253,0.00000,0.00000
254,0.00000,0.00000


,quality_5,quality_10
2505,10.00000,120.00000
2506,5.00000,60.00000
2507,0.00000,0.00000
2508,0.00000,0.00000
2509,0.00000,0.00000


,quality_6,quality_7
2505,4.00000,48.00000
2506,1.00000,12.00000
2507,0.00000,0.00000
2508,0.00000,0.00000
2509,0.00000,0.00000


,quality_11,quality_12
150,1.00000,12.00000
151,0.00000,0.00000
152,0.00000,0.00000
153,0.00000,0.00000
154,0.00000,0.00000


---

## 6-2. Fwver in quality data

* 먼저, Fwver의 대표적인 종류는 다음과 같다.
    - '03'으로 시작하는 펌웨어 버전
    - '04'으로 시작하는 펌웨어 버전
    - '05'으로 시작하는 펌웨어 버전
    - '09.17.1431' 특수한 펌웨어 버전

In [118]:
print(sorted(train_qual.fwver.unique()))

['03.11.1149', '03.11.1167', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3571', '04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778', '04.33.1125', '04.33.1149', '04.33.1185', '04.33.1261', '04.73.2237', '04.73.2571', '04.82.1684', '04.82.1778', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.66.3237', '05.66.3571', '09.17.1431']


###### 1. 펌웨어 버전에 따라서, 불만 제기 확률이 달라질까?

In [119]:
view_relation_fw_complain(train_qual)

100%|██████████| 27/27 [00:01<00:00, 24.85it/s]

펌웨어 버전 04.22.1684를 사용중인 사람들 가운데, 59.09%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 13명입니다.

펌웨어 버전 09.17.1431를 사용중인 사람들 가운데, 96.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 25명중 24명입니다.

펌웨어 버전 04.16.3571를 사용중인 사람들 가운데, 63.24%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 68명중 43명입니다.

펌웨어 버전 04.82.1684를 사용중인 사람들 가운데, 54.55%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 22명중 12명입니다.

펌웨어 버전 04.33.1149를 사용중인 사람들 가운데, 70.27%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 37명중 26명입니다.

펌웨어 버전 05.66.3237를 사용중인 사람들 가운데, 75.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 8명중 6명입니다.

펌웨어 버전 04.82.1778를 사용중인 사람들 가운데, 61.54%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 13명중 8명입니다.

펌웨어 버전 05.66.3571를 사용중인 사람들 가운데, 60.00%는 불만을 제기하였습니다. 
불만 제기를 한 사람의 수는 전체 5명중 3명입니다.



<b>특정 펌웨어 버전이 불만 제기에 유의미한 영향을 미침</b>임을 알 수 있다.<br>
→ &nbsp; 따라서 <span style="color:red">펌웨어 버전과 불만 제기 간의 관계</span>를 알 수 있는 파생변수를 만들어야 함을 알 수 있다.

---

# 7. Err Data & Quality Data 관계

---

### Err Data & Quality Data에서 중복되는 컬럼인 `Fwver`을 기준으로 먼저 생각해보기
&nbsp;&nbsp; → 두 Data에서 <u>모두 관측되는 Fwver</u>을 기준으로 두었다. (관계를 보기 위하여)

###### 1. Quality Data에서 -1을 제외한 0보다 크거나 같은 quality값을 보았을 경우 

In [120]:
def f2int(data):
    for i in data.columns:
        data[i] = data[i].apply(lambda x : int(x))
    return data

In [121]:
te_fwlst = list(train_err.fwver.unique())

for i in train_qual.columns[3:]:
    tq_fw = train_qual.groupby('fwver')[i].max().to_frame().reset_index()
    if i == 'quality_0':
        result_fw = tq_fw[tq_fw['fwver'].isin(te_fwlst)]
    else:
        tmp_2 = tq_fw[tq_fw['fwver'].isin(te_fwlst)].iloc[:,1]
        result_fw = pd.concat([result_fw, tmp_2], axis = 1)

In [122]:
result_fw

,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,03.11.1149,0,0,13,0,0,10,0,0,0,13,12,0,0
1,03.11.1167,0,171,191859,0,0,65221,500,5828,1317,315367,120367,14,14
2,04.16.3345,0,0,0,0,0,0,543,718,0,0,0,0,0
3,04.16.3439,0,1,0,0,0,1,5,5,1,0,2,0,0
4,04.16.3553,0,20,12089,0,0,127495,600,7200,42,12098,343052,4,4
5,04.16.3571,0,4,0,0,0,2082,94,124,7,0,4416,0,0
6,04.22.1442,0,0,0,0,0,3,0,0,0,0,8,0,0
7,04.22.1656,0,0,0,0,0,0,73,97,0,0,0,0,0
8,04.22.1666,0,1,0,0,0,531,0,0,1,0,636,0,0
9,04.22.1684,0,1,0,0,0,8698,600,3026,1,0,8841,0,0


<b>특정 펌웨어 버전이 quality columns와 유의미한 관계가 있지 않음</b>임을 알 수 있다.<br>
→ &nbsp; 그렇다면 <span style="color:red">펌웨어 버전을 포함하는 더 큰 개념인 Model</span>은 어떨까?

In [123]:
def change_fw_to_md(x):
    if x in ['04.22.1442', '04.22.1656', '04.22.1666', '04.22.1684', '04.22.1750', '04.22.1778']:
        return 'model_0'
    elif x in ['04.16.2641', '04.16.3345', '04.16.3439', '04.16.3553', '04.16.3569', '04.16.3571']:
        return 'model_1'
    elif x in ['04.33.1095', '04.33.1125', '04.33.1149', '04.33.1171', '04.33.1185', '04.33.1261']:
        return 'model_2'
    elif x in ['05.15.2090', '05.15.2092', '05.15.2114', '05.15.2120', '05.15.2122', '05.15.2138', '05.15.3104']:
        return 'model_3'
    elif x in ['03.11.1141', '03.11.1149', '03.11.1167']:
        return 'model_4'
    elif x in ['04.82.1684', '04.82.1730', '04.82.1778']:
        return 'model_5'
    elif x in ['10', '8.5.3']:
        return 'model_6'
    elif x in ['05.66.3237', '05.66.3571']:
        return 'model_7'
    elif x in ['04.73.2237', '04.73.2571']:
        return 'model_8'

In [124]:
result_fw.fwver = result_fw.fwver.apply(lambda x : change_fw_to_md(x))
fwver_dup = result_fw.groupby('fwver').sum()
fwver_dup.index.name = 'model_nm'
fwver_dup = f2int(fwver_dup)
fwver_dup

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
model_nm,,,,,,,,,,,,,
model_0,0,46,0,0,0,484834,1873,14094,91,0,2296572,12,15
model_1,0,25,12089,0,0,129578,1242,8047,50,12098,347470,4,4
model_2,0,39,0,0,0,281411,1840,12808,59,0,1699692,14,21
model_3,157667,20,157667,0,0,638366,955,7662,73,397424,1912507,6,10
model_4,0,171,191872,0,0,65231,500,5828,1317,315380,120379,14,14
model_5,0,7,0,0,0,189,610,2576,9,0,740,0,0
model_7,0,4,0,0,0,304,317,337,4,0,437,0,0
model_8,0,1,0,0,0,9698,51,51,4,0,9698,0,0


이 역시 Fwver과 마찬가지로 <b>quality columns와 유의미한 관계가 있지 않음</b>임을 알 수 있다.<br>

###### 2. Quality값에서 언급하였던 <span style="color:blue"> -1값</span>만을 보았을 경우 

In [125]:
minus_qual = pd.read_csv(save_path+"train_quality_data_time.csv")

In [126]:
for i in minus_qual.columns[3:]:
    idx = minus_qual[minus_qual[i] != -1].index
    minus_qual.loc[idx,i] = 0 # -1이 아닌 값을 모두 0으로 교체

In [127]:
te_fwlst = list(train_err.fwver.unique())

for i in minus_qual.columns[3:]:
    tq_fw = minus_qual.groupby('fwver')[i].sum().to_frame().reset_index()
    if i == 'quality_0':
        restmp = tq_fw[tq_fw['fwver'].isin(te_fwlst)]
    else:
        tmp_2 = tq_fw[tq_fw['fwver'].isin(te_fwlst)].iloc[:,1]
        restmp = pd.concat([restmp, tmp_2], axis = 1)

In [128]:
restmp.fwver = restmp.fwver.apply(lambda x : change_fw_to_md(x))
restmp = restmp.groupby('fwver').sum()
restmp.index.name = 'model_nm'
restmp = f2int(restmp)
restmp

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
model_nm,,,,,,,,,,,,,
model_0,-38743,-38743,-38743,0,0,-38742,-38743,0,0,0,0,-38743,0
model_1,-19731,-19730,-19731,0,0,-19731,-19730,0,0,0,0,-19730,0
model_2,-41207,-41207,-41207,0,0,-41207,-41207,0,0,0,0,-41207,0
model_3,-29560,-29560,-29560,0,0,-29560,-29560,0,0,0,0,-29560,0
model_4,0,-13564,-13564,0,0,-13564,-13564,0,0,0,0,-13564,0
model_5,-314,-314,-314,0,0,-314,-314,0,0,0,0,-314,0
model_7,-170,-170,-170,0,0,-170,-170,0,0,0,0,-170,0
model_8,-43,-43,-43,0,0,-43,-43,0,0,0,0,-43,0


---

### Fwver을 Err Data에 있는 Model에 해당하는 카테고리로 변경하였을 경우에는 <span style="color:red">유의미한 결과</span>를 확인할 수 있었다.  
&nbsp;→ Model_4 & Quality_0과 Model_1 & Quality_1을 제외하고 모든 데이터프레임 안의 값이 같다는 사실을 확인할 수 있다.  

&nbsp;→ &nbsp;`-1값`은 Quality_0 & Quality_1 & Quality_2 & Qualiy_5 & Quality_6 & Quality_11&nbsp;에서 모두 (합이) 같게 측정된다.  
&nbsp;→ &nbsp;Quality_0 & Quality_1 & Quality_2 & Qualiy_5 & Quality_6 & Quality_11&nbsp;는 `-1값`을 측정할 때 같이 움직인다.

### ⇒ 그러므로, Quality 수치 해석에서 다루었던 `-1`을 따로 보는 것은 유의미하다.

<center><span style="color:gray">※ 참고 : Model_4 + Quality_0에서 -13564이 아닌 0값이 측정된 이유는 Fwver 결측치를 제거하는 과정에서 삭제된 것으로 예상된다.<br>따라서 Fwver 결측치는 13564개의 -1로 대체하는 것이 바람직하다는 결론을 내릴 수 있다.</span></center>

---

# 8. 결과 정리

---

## Err Data & Quality Data를 분석해본 결과 정리

#### 1. `Errtype & Errcode`가 불만 제기에 유의미한 변수였으며, 각 종류별 Err 발생 횟수와 이를 단위 시간별로 나누어 보는 것이 중요하다.
#### 2. `Fwver`과 `Model`은 포함관계를 갖고 있으며, 불만 제기에 유의미한 변수는 `Fwver`이었다.
#### 3. 사용자들은 `Fwver`과 `Model`의 변화에 민감하게 반응하였음을 알 수 있다.
#### 4. `Quality Log`의 수치들은 -1과 나머지로 먼저 나누어 볼 수 있고, 특히 -1은 불만 제기에 유의미한 영향을 미친다.
#### 5. `Quality` 칼럼들 간의 관계성이 있음을 입증할 수 있었고, `Quality` 데이터의 통계적 분석(최대값, 평균, 표준편차, 이동평균 등)도 중요할 것으로 예상된다.
#### 6. Err Data의 `Model`과 Quality Data의 `Quality 측정값`의 데이터프레임을 보면, `특정 모델별` 측정되는 `Quality` 의 `관계`를 확인할 수 있다.

---